# Init

In [74]:
import duckdb
import pandas as pd
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging    

In [46]:
cd ..

/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab


/Users/wliao0504/.pyenv/versions/3.13.0/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [47]:
pwd

'/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab'

In [60]:
%load_ext autoreload
%autoreload 2
from src.utils import check_duplicates, find_duplicates, read_from_rclif, clif_table_pathfinder, \
    mimic_table_pathfinder

from importlib import reload
import src.tables.respiratory_support
reload(src.tables.respiratory_support)
from src.tables.respiratory_support import RespPipeline
# reload(src.tables.respiratory_support.RespPipeline)

2025-03-28 09:16:43,715 - INFO - initialized logging at logs/etl.log


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dev

In [5]:
clif_resp = read_from_rclif("respiratory_support")

## Check all-NA rows

In [81]:
clif_resp.head(), len(clif_resp)

(  hospitalization_id       recorded_dttm        device_name device_category  \
 0           20000094 2150-03-02 15:34:00      Nasal cannula   Nasal Cannula   
 1           20000094 2150-03-02 20:00:00      Nasal cannula   Nasal Cannula   
 2           20000094 2150-03-03 05:53:00               None            None   
 3           20000094 2150-03-03 08:00:00               None            None   
 4           20000147 2121-08-30 22:00:00  Endotracheal tube             IMV   
 
   vent_brand_name            mode_name                  mode_category  \
 0            None                 None                           None   
 1            None                 None                           None   
 2            None                 None                           None   
 3            None                 None                           None   
 4            None  CMV/ASSIST/AutoFlow  Assist Control-Volume Control   
 
    tracheostomy  fio2_set  lpm_set  ...  inspiratory_time_set  peep_set

In [ ]:
# create a new column called "all_value_na" if all the columns starting from "tracheostomy" are NA
clif_resp["all_value_na"] = clif_resp.loc[:, "fio2_set": "mean_airway_pressure_obs"].isna().all(axis=1)
clif_resp.head()

,hospitalization_id,recorded_dttm,device_name,device_category,vent_brand_name,mode_name,mode_category,tracheostomy,fio2_set,lpm_set,...,inspiratory_time_set,peep_set,tidal_volume_obs,resp_rate_obs,plateau_pressure_obs,peak_inspiratory_pressure_obs,peep_obs,minute_vent_obs,mean_airway_pressure_obs,all_value_na
0,20000094,2150-03-02 15:34:00,Nasal cannula,Nasal Cannula,None,None,None,False,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,20000094,2150-03-02 20:00:00,Nasal cannula,Nasal Cannula,None,None,None,False,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,20000094,2150-03-03 05:53:00,None,None,None,None,None,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,20000094,2150-03-03 08:00:00,None,None,None,None,None,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,20000147,2121-08-30 22:00:00,Endotracheal tube,IMV,None,CMV/ASSIST/AutoFlow,Assist Control-Volume Control,False,1.0,NaN,...,0.9,5.0,524.0,16.0,23.0,25.0,NaN,9.77,11.0,False


In [11]:
# a smaller subsect of important columns to inspect
resp2 = clif_resp[["hospitalization_id", "recorded_dttm", "device_name", "vent_brand_name", "mode_name", "all_value_na"]]
resp2.head()

,hospitalization_id,recorded_dttm,device_name,vent_brand_name,mode_name,all_value_na
0,20000094,2150-03-02 15:34:00,Nasal cannula,None,None,False
1,20000094,2150-03-02 20:00:00,Nasal cannula,None,None,False
2,20000094,2150-03-03 05:53:00,None,None,None,True
3,20000094,2150-03-03 08:00:00,None,None,None,True
4,20000147,2121-08-30 22:00:00,Endotracheal tube,None,CMV/ASSIST/AutoFlow,False


The result does seem to show that a lot rows are completely uninformative. let's go back to the source data and see what it was like originally.

In [ ]:
pipeline = RespPipeline()
pipeline.extract()

In [39]:
src_df = pipeline.data["resp_events"]
src_df.head()

,itemid,label,hadm_id,stay_id,time,value,valueuom,variable
0,223834,O2 Flow,23251352,31494479,2141-12-18 00:00:00,4,L/min,lpm_set
1,226732,O2 Delivery Device(s),23251352,31494479,2141-12-18 00:00:00,Nasal cannula,None,device_name
2,223834,O2 Flow,23251352,31494479,2141-12-18 03:00:00,3,L/min,lpm_set
3,226732,O2 Delivery Device(s),23251352,31494479,2141-12-18 03:00:00,Nasal cannula,None,device_name
4,223834,O2 Flow,23251352,31494479,2141-12-18 04:00:00,2,L/min,lpm_set


start with this: 20000094	2150-03-03 05:53:00

In [40]:
src_df.loc[
    (src_df["hadm_id"] == 20000094) & (src_df["time"] == pd.to_datetime("2150-03-03 05:53:00")),
    ]

,itemid,label,hadm_id,stay_id,time,value,valueuom,variable
5628471,226732,O2 Delivery Device(s),20000094,35605481,2150-03-03 05:53:00,None,None,device_name


So now check the initial mapping and see if we might have included redundant stuff:

In [41]:
pipeline.resp_mapper

{226732: 'device_name',
 223835: 'fio2_set',
 220339: 'peep_set',
 224685: 'tidal_volume_obs',
 224687: 'minute_vent_obs',
 224697: 'mean_airway_pressure_obs',
 224695: 'peak_inspiratory_pressure_obs',
 223834: 'lpm_set',
 223848: 'vent_brand_name',
 224690: 'resp_rate_obs',
 223849: 'mode_name',
 224688: 'resp_rate_set',
 224684: 'tidal_volume_set',
 224738: 'inspiratory_time_set',
 224686: 'tidal_volume_obs',
 224701: 'pressure_support_set',
 224696: 'plateau_pressure_obs',
 229314: 'mode_name',
 224691: 'flow_rate_set',
 224700: 'peep_obs',
 225448: 'tracheostomy',
 226237: 'tracheostomy',
 227287: 'lpm_set',
 224422: 'resp_rate_obs',
 224421: 'tidal_volume_obs',
 227577: 'mode_name',
 227579: 'peep_set',
 227580: 'peak_inspiratory_pressure_set',
 227582: 'flow_rate_set',
 224702: 'pressure_control_set',
 227581: 'resp_rate_set',
 None: None,
 'None': None}

In [30]:
pipeline.resp_mode_mapper

{'CPAP/PSV': 'Pressure Support/CPAP',
 'CMV/ASSIST/AutoFlow': 'Assist Control-Volume Control',
 'CMV/ASSIST': 'Assist Control-Volume Control',
 'MMV/PSV/AutoFlow': 'Other',
 'Standby': None,
 'PSV/SBT': 'Pressure Support/CPAP',
 'PRVC/AC': 'Pressure-Regulated Volume Control',
 'PCV+Assist': 'Pressure Control',
 'CPAP/PSV+ApnVol': 'Pressure-Regulated Volume Control',
 'SIMV/PSV/AutoFlow': 'SIMV',
 'MMV/PSV': 'Other',
 'CPAP': 'Pressure Support/CPAP',
 'APRV': 'Other',
 'CMV/AutoFlow': 'Assist Control-Volume Control',
 'CPAP/PPS': 'Pressure Support/CPAP',
 'CPAP/PSV+ApnPres': 'Pressure Support/CPAP',
 'VOL/AC': 'Assist Control-Volume Control',
 'PCV+': 'Pressure Control',
 'CMV': 'Assist Control-Volume Control',
 'PCV+/PSV': 'Pressure Control',
 'PRES/AC': 'Assist Control-Volume Control',
 'SIMV/PSV': 'SIMV',
 'APV (cmv)': 'Pressure-Regulated Volume Control',
 'SPONT': 'Pressure Support/CPAP',
 'MMV': 'Other',
 'Apnea Ventilation': 'Blow by',
 'APRV/Biphasic+ApnVol': 'Other',
 'MMV/AutoF

In [33]:
pipeline.resp_device_mapper

{'Endotracheal tube': 'IMV',
 'Nasal cannula': 'Nasal Cannula',
 'Tracheostomy tube': 'IMV',
 'Aerosol-cool': 'Face Mask',
 'Face tent': 'Face Mask',
 'Trach mask': 'Trach Collar',
 'High flow nasal cannula': 'High Flow NC',
 'Bipap mask': 'NIPPV',
 'High flow neb': 'High Flow NC',
 'Non-rebreather': 'Face Mask',
 'CPAP mask': 'CPAP',
 'Venti mask': 'Face Mask',
 'Other': 'Other',
 'Medium conc mask': 'Face Mask',
 'Oxymizer': 'Face Mask',
 'T-piece': 'Other',
 'Ultrasonic neb': 'Other',
 'Vapomist': 'Other',
 nan: nan,
 None: None,
 'None': None}

so the issue seem to be with only `resp_mapper`

In [32]:
resp_mapping = load_mapping_csv("respiratory_support")
resp_mapping.head()

,variable,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances,note / comment,status,latest update date,Unnamed: 13,variables,count.1,itemid_prio,label_prio,note
0,MAPPED ELSEWHERE,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,6393762,"Max: 2355560.0, Min: 0.0, Mean: 20.45996459206...",mapped in vitals,coded/validated by K. Smith,"July 2, 2024",NaN,tracheostomy,2.0,225448.0,Percutaneous Tracheostomy,likely no conflict
1,MAPPED ELSEWHERE,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,6324341,"Max: 9900000.0, Min: -272.0, Mean: 104.8276359...",mapped in vitals,coded/validated by K. Smith,"July 2, 2024",NaN,fio2_set,1.0,NaN,NaN,NaN
2,device_name,226732,O2 Delivery Device(s),O2 Delivery Device(s),chartevents,Respiratory,NaN,Text,1536870,"""{'Endotracheal tube': 467931, 'Nasal cannula'...",NaN,coded/validated by K. Smith,"July 2, 2024",NaN,lpm_set,2.0,223834.0,O2 Flow,NaN
3,fio2_set,223835,Inspired O2 Fraction,FiO2,chartevents,Respiratory,NaN,Numeric,849875,"Max: 10050.0, Min: 0.0, Mean: 48.94540639505812",NaN,coded/validated by K. Smith,"July 2, 2024",NaN,tidal_volume_set,1.0,NaN,NaN,NaN
4,peep_set,220339,PEEP set,PEEP set,chartevents,Respiratory,cmH2O,Numeric,642089,"Max: 8774580.0, Min: -5.0, Mean: 26.2141163608...",NaN,coded/validated by K. Smith,"July 2, 2024",NaN,tidal_volume_obs,3.0,224685.0,Tidal Volume (observed),NaN


In [34]:
resp_device_mapping = load_mapping_csv("device_category")
resp_mode_mapping = load_mapping_csv("mode_category")

resp_mapper = construct_mapper_dict(
    resp_mapping, 
    key_col = "itemid", 
    value_col = "variable", 
    map_none_to_none = True,
    decision_col = "variable"
    )
resp_device_mapper = construct_mapper_dict(
    mapping_df = resp_device_mapping, 
    key_col = "device_name", 
    value_col = "device_category", 
    map_none_to_none = True,
    excluded_item_ids = ["223848"],
    decision_col = "device_category"
    )
resp_mode_mapper = construct_mapper_dict(
    mapping_df = resp_mode_mapping, 
    key_col = "mode_name", 
    value_col = "mode_category", 
    map_none_to_none = False,
    decision_col = "mode_category"
    )

In [36]:
resp_mapper, resp_device_mapper, resp_mode_mapper

({226732: 'device_name',
  223835: 'fio2_set',
  220339: 'peep_set',
  224685: 'tidal_volume_obs',
  224687: 'minute_vent_obs',
  224697: 'mean_airway_pressure_obs',
  224695: 'peak_inspiratory_pressure_obs',
  223834: 'lpm_set',
  223848: 'vent_brand_name',
  224690: 'resp_rate_obs',
  223849: 'mode_name',
  224688: 'resp_rate_set',
  224684: 'tidal_volume_set',
  224738: 'inspiratory_time_set',
  224686: 'tidal_volume_obs',
  224701: 'pressure_support_set',
  224696: 'plateau_pressure_obs',
  229314: 'mode_name',
  224691: 'flow_rate_set',
  224700: 'peep_obs',
  225448: 'tracheostomy',
  226237: 'tracheostomy',
  227287: 'lpm_set',
  224422: 'resp_rate_obs',
  224421: 'tidal_volume_obs',
  227577: 'mode_name',
  227579: 'peep_set',
  227580: 'peak_inspiratory_pressure_set',
  227582: 'flow_rate_set',
  224702: 'pressure_control_set',
  227581: 'resp_rate_set',
  None: None,
  'None': None},
 {'Endotracheal tube': 'IMV',
  'Nasal cannula': 'Nasal Cannula',
  'Tracheostomy tube': 'IMV

In [65]:
p2 = RespPipeline()
p2.extract()

2025-03-28 09:24:24,821 - INFO - initialized logging at logs/etl.log
2025-03-28 09:24:24,881 - INFO - starting to build clif respiratory support table -- extracting source data from MIMIC-IV...
2025-03-28 09:24:24,885 - INFO - parsing the mapping files to identify relevant items and fetch corresponding events...
2025-03-28 09:24:24,885 - INFO - querying the d_items table to identify which event tables to be separately queried for 31 items
2025-03-28 09:24:24,887 - INFO - identified 2 event tables to be separately queried: ['chartevents', 'procedureevents']
2025-03-28 09:24:24,887 - INFO - fetching events from chartevents table for 29 items


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-03-28 09:24:30,226 - INFO - fetched 13719827 events from chartevents table for 29 items
2025-03-28 09:24:30,227 - INFO - fetching events from procedureevents table for 2 items
2025-03-28 09:24:30,239 - INFO - fetched 680 events from procedureevents table for 2 items
2025-03-28 09:24:30,454 - INFO - concatenated 13720507 events from 2 event tables


In [67]:
df_c = p2.transform()

2025-03-28 09:26:02,977 - INFO - identified 127425 'duplicated' events to be cleaned.
2025-03-28 09:26:02,977 - INFO - removing the first type of duplicates: lower-ranked 'duplicated' devices...
2025-03-28 09:26:05,756 - INFO - removing the second type of duplicates: duplicated device reads...
2025-03-28 09:26:09,724 - INFO - pivoting to a wide format and coalescing duplicate columns...
/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC/src/tables/respiratory_support.py:152: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resp_wider_in_ids["lpm_set"] = resp_wider_in_ids[223834].fillna(resp_wider_in_ids[227287])
2025-03-28 09:26:16,140 - INFO - mapping device and mode names to categories...
2025-03-28 09:26:17,130 - INFO - renaming, reord

In [82]:
df_c

itemid,hospitalization_id,recorded_dttm,device_name,device_category,vent_brand_name,mode_name,mode_category,tracheostomy,fio2_set,lpm_set,...,peak_inspiratory_pressure_set,inspiratory_time_set,peep_set,tidal_volume_obs,resp_rate_obs,plateau_pressure_obs,peak_inspiratory_pressure_obs,peep_obs,minute_vent_obs,mean_airway_pressure_obs
0,20000094,2150-03-02 15:34:00,Nasal cannula,Nasal Cannula,NaN,NaN,NaN,False,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000094,2150-03-02 20:00:00,Nasal cannula,Nasal Cannula,NaN,NaN,NaN,False,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000094,2150-03-03 05:53:00,None,None,NaN,NaN,NaN,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000094,2150-03-03 08:00:00,None,None,NaN,NaN,NaN,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20000147,2121-08-30 22:00:00,Endotracheal tube,IMV,NaN,CMV/ASSIST/AutoFlow,Assist Control-Volume Control,False,100.0,NaN,...,NaN,0.9,5.0,524.0,16.0,23.0,25.0,NaN,9.77,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542624,29999828,2180-10-29 00:00:00,Nasal cannula,Nasal Cannula,NaN,NaN,NaN,False,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2542625,29999828,2180-10-29 04:00:00,Nasal cannula,Nasal Cannula,NaN,NaN,NaN,False,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2542626,29999828,2180-10-29 08:00:00,Nasal cannula,Nasal Cannula,NaN,NaN,NaN,False,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2542627,29999828,2180-10-29 08:36:00,Nasal cannula,Nasal Cannula,NaN,NaN,NaN,False,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
p2.resp_mapper, p2.resp_device_mapper, p2.resp_mode_mapper

({226732: 'device_name',
  223835: 'fio2_set',
  220339: 'peep_set',
  224685: 'tidal_volume_obs',
  224687: 'minute_vent_obs',
  224697: 'mean_airway_pressure_obs',
  224695: 'peak_inspiratory_pressure_obs',
  223834: 'lpm_set',
  223848: 'vent_brand_name',
  224690: 'resp_rate_obs',
  223849: 'mode_name',
  224688: 'resp_rate_set',
  224684: 'tidal_volume_set',
  224738: 'inspiratory_time_set',
  224686: 'tidal_volume_obs',
  224701: 'pressure_support_set',
  224696: 'plateau_pressure_obs',
  229314: 'mode_name',
  224691: 'flow_rate_set',
  224700: 'peep_obs',
  225448: 'tracheostomy',
  226237: 'tracheostomy',
  227287: 'lpm_set',
  224422: 'resp_rate_obs',
  224421: 'tidal_volume_obs',
  227577: 'mode_name',
  227579: 'peep_set',
  227580: 'peak_inspiratory_pressure_set',
  227582: 'flow_rate_set',
  224702: 'pressure_control_set',
  227581: 'resp_rate_set',
  None: None,
  'None': None},
 {'Endotracheal tube': 'IMV',
  'Nasal cannula': 'Nasal Cannula',
  'Tracheostomy tube': 'IMV

In [73]:
src_df.loc[
    (src_df["hadm_id"] == 20000094),
    ]

,itemid,label,hadm_id,stay_id,time,value,valueuom,variable
5628466,223834,O2 Flow,20000094,35605481,2150-03-02 15:34:00,4,L/min,lpm_set
5628467,226732,O2 Delivery Device(s),20000094,35605481,2150-03-02 15:34:00,Nasal cannula,None,device_name
5628468,226732,O2 Delivery Device(s),20000094,35605481,2150-03-03 08:00:00,None,None,device_name
5628469,223834,O2 Flow,20000094,35605481,2150-03-02 20:00:00,4,L/min,lpm_set
5628470,226732,O2 Delivery Device(s),20000094,35605481,2150-03-02 20:00:00,Nasal cannula,None,device_name
5628471,226732,O2 Delivery Device(s),20000094,35605481,2150-03-03 05:53:00,None,None,device_name


let's also double check against the real source, which shows they were indeed NA despite having a corresponding itemid (so a device plugged in). focus on 20000094	2150-03-03 05:53:00

In [80]:
query = f"""
SELECT *
FROM '{mimic_table_pathfinder("chartevents")}'
WHERE hadm_id = 20000094
    AND itemid IN (223834, 226732)
"""
src_df2 = duckdb.query(query).df()
src_df2.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,14046553,20000094,35605481,19825,2150-03-02 15:34:00,2150-03-02 15:34:00,223834,4,4.0,L/min,0
1,14046553,20000094,35605481,97419,2150-03-02 20:00:00,2150-03-02 20:15:00,223834,4,4.0,L/min,0
2,14046553,20000094,35605481,19825,2150-03-02 15:34:00,2150-03-02 15:34:00,226732,Nasal cannula,NaN,None,0
3,14046553,20000094,35605481,31521,2150-03-03 08:00:00,2150-03-03 08:03:00,226732,None,NaN,None,0
4,14046553,20000094,35605481,97419,2150-03-02 20:00:00,2150-03-02 20:15:00,226732,Nasal cannula,NaN,None,0


In [83]:
src_df = p2.data["resp_events"]
src_df.loc[
    (src_df["hadm_id"] == 20000094) & (src_df["time"] == pd.to_datetime("2150-03-03 05:53:00")),
    ]

,itemid,label,hadm_id,stay_id,time,value,valueuom,variable
5598603,226732,O2 Delivery Device(s),20000094,35605481,2150-03-03 05:53:00,None,None,device_name


This seems quite weird b/c this original row was not found in the final df